# The Routes project


# Installations and the program code

In this project our main goal is to group locations (addresses) to different routes
Such that the average aerial distance between these routes will be as minimal as possible

In [1]:
# what to install
!pip install geopy
!pip install folium
!pip install chardet

Importing the data from a csv file

In [2]:
import requests
url = 'https://raw.githubusercontent.com/danihello/Routes/main/people_Un.csv'
response = requests.get(url)
content = response.text
print(content)

fname,lname,age,gender,country,city,street
assaf,hazan,30,m,ישראל,tel aviv,beit hilel 1
daniel,premisler,33,m,ישראל,פתח תקווה,אחד העם 17
adam,efrati,34,m,ישראל,בני ברק,דב גרונר 2
serj,itzhakov,34,m,ישראל,תל אביב,אירוס 2
alon,shomron,32,m,ישראל,ראשון לציון,כצנלסון 26
jenia,premisler,60,m,ישראל,ראשון לציון,הנורית 1
olla,premisler,38,f,ישראל,חיפה,קרן היסוד
dana,saar,25,f,ישראל,קדימה,ההסתדרות 3
noam,saar,30,m,ישראל,נתניה,מוריה 5



In [3]:
with open('people_Un.csv', 'w') as f:
    f.write(content)

Main code: Person, Address and Route classes

In [4]:
# main code
import chardet
import csv
import folium
import time
from geopy import Nominatim
from geopy import distance
from itertools import combinations

class Person():
    """
    A class used to represent a Person

    Attributes
    ----------
    fname : str
        first name of a person
    lname : str
        last name of a person
    fullname: str
        full name of a person
    age: int
        age in years of a person
    gender: str
        a gender of a person
    address: Address
        the address of of a person - can be created using country, city and street
        or an existing address
    
    Methods:
    --------
    display(tooltip=None, popup=None, c=None)
        shows the adress of a person on a map with his full name as a tool tip
        and his full adress as a popup
    get_distance(destination)
        return the aerial distance (in km) between a person to another
        person or an address or a string that represents a valid address
    """
    def __init__(self, fname, lname, age, gender, country=None, city=None, street=None, address=None):
        self.fname = fname.strip().title()
        self.lname = lname.strip().title()
        self.fullname = fname.strip().title() + ' ' + lname.strip().title()
        self.age = int(age)
        self.gender = gender.strip().upper()
        if isinstance(address, Address):
            self.address=address
        else:
            self.address = Address(country.strip(), city.strip(), street.strip())
    
    def __str__(self):
        return " ,".join([self.fullname, str(self.age), self.gender])
    
    def __repr__(self):
        return " ,".join([self.fullname, str(self.age), self.gender])
    
    def display(self, tooltip=None, popup=None, c=None, m=None, zoom=15):
        """returns a map object with a marker of a person's address
        Args:
            tooltip (str): the text for the marker's tooltip (defualt is None)
            popup (str): the text for the marker's popup (default is None)
            c (str): the color of the marker (default is None)

        Returns:
            Map: a map object with a marker of the person address coordinates
        """
        if m is None:
            m = folium.Map(location = [*self.address.coord], zoom_start=zoom)
        if tooltip is None:
            tooltip = self.fullname
        if popup is None:
            popup = self.address.full_address
        if c is None:
            c = 'blue'
        folium.Marker(location=self.address.coord, 
              tooltip=tooltip, 
              popup=folium.Popup(popup, max_width=len(popup)*10),
              icon=folium.Icon(color=c)).add_to(m)
        return m
    
    def get_distance(self, destination):
        """returns aerial distance between two locations in km
        Args:
            destination (str, Address, Person)
        Returns:
            float: distance in km
        """
        if isinstance(destination, str):
            location = Address(full_address_str=destination)
        elif isinstance(destination, Person):
            location = destination.address
        else:
            location = destination
        return Address.strt_line_distance(self.address, location)

class Address():
    """
    A class used to represent an Address

    Parameters
    ----------
    country : str
        country of the address (default is None)
    city : str
        city of the address (default is None)
    street : str
        street/road of the address (default is None)
    full_address_str : str
        text that represent the full address (default is None)
    
    Attributes
    ----------
    city : str
        city of the address
    country_code : str
        country code of the address
    country : str
        country of the address
    street : str
        street part of the address
    latitude : float
        the address latitude value
    longitude :float
        the address longitude value
    coord : tuple
        tuple of latitude, longitude
    full_address : str
        text that represents the full address 
    
    Raises:
    --------
        TypeError : if no full_address_str was supplied and country/city/street are missing

    Methods:
    --------
    display(tooltip=None, popup=None, c=None)
        shows the adress of a person on a map with his full name as a tool tip
        and his full adress as a popup
    get_distance(destination)
        return the aerial distance (in km) between a person to another
        person or an address or a string that represents a valid address
    """
    def __init__(self, country=None, city=None, street=None, full_address_str=None):
        if full_address_str is None:
            try:
                address_str = " ,".join([country, city, street])
            except TypeError:
                raise TypeError("missing parameters: country, city, street")
        else:
            address_str = full_address_str
        self.parse_address(address_str)
        self.full_address = " ,".join([self.country, self.city, self.street])
    
    def __str__(self):
        return self.full_address
    
    def __repr__(self):
        return self.full_address
    
    def __eq__(self, other):
        if self.coord == other.coord:
            return True
        return False
    
    def __ne__(self, other):
        if self.coord != other.coord:
            return True
        return False
    
    def __hash__(self):
        return hash(self.coord)
    
    def parse_address(self, address_str):
        """tries to parse the address (geocode) and sets the attributes of the class
        Args:
            address_str (str): represents a full address

        Raises:
            ValueError: if the address couldn't be parsed
        """
        locator = Nominatim(user_agent="myGeocoder")
        location = locator.geocode(address_str, language='en', addressdetails=True, timeout=10)
        if location is None:
            raise ValueError(f'Couldnt parse address {address_str}')
        address_dict = location.raw['address']
        self.city = address_dict.get('city',address_dict.get('town',''))
        self.country_code = address_dict.get('country_code')
        self.country = address_dict.get('country')
        self.street = " ".join([address_dict.get('road'), address_dict.get('house_number','')])
        self.latitude = float(location.raw['lat'])
        self.longitude = float(location.raw['lon'])
        self.coord = self.latitude , self.longitude
        
    def display(self, m=None, zoom=10): 
        """returns a map object with a marker of the address
           if a map object was supplied the method will return the map with an added marker
        Args:
            m (Map): the text for the marker's tooltip (defualt is None)
            zoom (int): zoom value to the map (default is 10)

        Returns:
            Map: a map object with a marker of the person address coordinates
        """
        if m is None:
            m = folium.Map(self.coord, zoom_start=zoom)
        folium.Marker(location=self.coord, 
                        tooltip=self.full_address, 
                        popup=f'coordinates: {self.coord}',
                        icon=folium.Icon(color='blue')).add_to(m)
        return m
    
    def strt_line_distance(self, destination):
        """returns aerial distance between two locations in km
        Args:
            destination (Address)
        Returns:
            float: distance in km
        """
        return distance.distance(self.coord, destination.coord).km
    
class Route():
    """
    A class used to represent a Route

    Parameters
    ----------
    *people : 
        list of people, forming the route
    
    Attributes
    ----------
    locations : Dict(Person: Address)
        a dictionary of people and their addresses
    distance : float
        the total distance between all the combinations of the locations
        for example for locations (a, b, c)  the total distance would be the sum of distances
        of these locations:
        a --> b
        a --> c
        b --> c
    country : str
        country of the address
    street : str
        street part of the address
    latitude : float
        the address latitude value
    longitude :float
        the address longitude value
    coord : tuple
        tuple of latitude, longitude
    full_address : str
        text that represents the full address 
    
    Raises:
    --------
        IndexError : if *people len is less then 2 

    Methods:
    --------
    conjoint_location(other)
        returns True if both routes has have a conjoint address else returns False
    location_count()
        returns the count of locations in a route
    display(zoom=10, map=None)
        returns a map object with markers of the map locations and polylines between all locations
    """
    def __init__(self, *people):
        if len(people) < 2:
            raise IndexError('route requires at least two people')
        self.locations = {person: person.address for person in people}
        self.distance = sum([distance.distance(adress1.coord, adress2.coord).km 
                             for adress1, adress2 in combinations(self.locations.values(), 2)])
        
    def __eq__(self, other):
        if set(self.locations) == set(other.locations):
            return True
        else:
            return False

    def __ne__(self, other):
        if set(self.locations) != set(other.locations):
            return True
        else:
            return False
        
    def __str__(self):
        addresses = list(self.locations.values())
        addresses_str = [address.full_address for address in addresses]
        persons_str = [person.__str__() for person in self.locations]
        info = zip(persons_str, addresses_str)
        info_str = [f"personal info: {person[0]:<20} ; address: {person[1]:<20}" for person in info]
        return f'numer of locations: {len(info_str)}' + '\n' + '\n'.join(info_str)
    
    def __repr__(self):
        addresses = list(self.locations.values())
        addresses_str = [address.full_address for address in addresses]
        persons_str = [person.__str__() for person in self.locations]
        info = zip(persons_str, addresses_str)
        info_str = ['personal info: ' + person[0] + ' | ' + 'address: ' + person[1] for person in info]
        return f'numer of locations: {len(info_str)}' + '\n' + '\n'.join(info_str) + '\n'

    def conjoint_location(self, other):
        """returns True if both routes have a conjoin location
        Args:
            other (Route): another route
        Returns:
            bool: returns True if both routes have a conjoin address, else returns False
        """
        if set(self.locations) & set(other.locations):
            return True
        return False                
    
    def location_count(self):
        """returns the count of locations in a route
        """
        return len(self.locations)
        
    def display(self, zoom=10, m=None):
        """returns a map object with markers of the map locations and polylines between all locations.
        if an m parameter is passed the method will add the route to the map object

        Args:
            zoom (int): the initial zoom to the map object (defualts to 10)
            m (Map): a map object to add the route to (defaults to None)

        Returns:
            Map:  returns a map object with markers of the map locations and polylines between all locations
        """
        start_location = [point.coord for point in self.locations.values()][0]
        if m is None:
            m = folium.Map(start_location, zoom_start=zoom)
        points = [point.coord for point in self.locations.values()] 
        for pair in combinations(points, 2):
            distance_km = round(distance.distance(pair[0], pair[1]).km,2)
            folium.PolyLine(pair, 
                            tooltip='click to find the distance',
                            popup=f'{distance_km} km').add_to(m)
        for person in self.locations:
            popup = person.address.full_address
            folium.Marker(location=person.address.coord, 
                          tooltip=person.fullname, 
                          popup=folium.Popup(popup, max_width=len(popup)*10),
                          icon=folium.Icon(color='blue')).add_to(m)
        return m
    

def get_people_from_csv(filename):
    """gets a csv file and returns a list of people

    Args:
        filename path to a csv file

    Returns:
        list(People): returns a list of persons with address and personal information
    """
    people = []
    #we use chardet to find the correct encoding for the file
    with open(filename, 'rb') as csvfile:
        content = csvfile.read()
    det = chardet.detect(content)
    encoder = det['encoding']
    #we can now read the file using the correct encoding
    with open(filename, encoding=encoder) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            people.append(Person(**row))
    return people

def get_people_from_text(text):
    lines = text.splitlines()
    columns = lines[0].split(',')
    people = []
    for line in lines[1:]:
        data = line.split(',')
        personal_data = {columns[i]:data[i] for i in range(len(columns))}
        people.append(Person(**personal_data))
    return people

def get_routes(people, per_route=2):
    """creates a list of routes

    Args:
        per_route (int): number of different people per a route (default is 2)

    Returns:
        list(Route): returns a list of different combinations of routes from the argument people
    """
    routes = []
    for group in combinations(people, per_route):
            route = Route(*group)
            routes.append(route)
    return routes

def legal_route(routes):
    """checks if the routes passed as an argument does not have the same locations and if they are not equal

    Args:
        routes (list): a list of routes 
    Returns:
        bool: returns True if the routes are not the same and have no conjoin location, else returns False
    """
    for r1 in routes:
        for r2 in routes:
            if r1.conjoint_location(r2) and r1 != r2:
                return False
    return True

def make_groups(people):
    """create a list of tuples which consists of routes such that every argument
    in the list will containt the people passed in the first agrument splited to routes
    
    Args:
        people (list): a list of people (Person class)
    
    Returns:
        list: returns a list of tuples of routes.
              each argument consists of a tuple of the people splitted to different Routes
              if the length of people is event all the routes would consist of 2 locations
              else one route will have 3 locations
    """
    routes_combos = []
    n_people = len(people)
    splited_routes = []
    duos = []
    if n_people % 2 == 0:
        num_of_splits = n_people // 2
        routes = get_routes(people)
        for arg in combinations(routes, num_of_splits):
            if legal_route(arg):
                splited_routes.append(arg)
        return splited_routes
    else:
        num_of_splits = n_people // 2 - 1
        routes_2 = get_routes(people)
        routes_3 = get_routes(people, 3)
        for duos in combinations(routes_2, num_of_splits):
            for y in routes_3:
                l_temp = list(duos)
                l_temp.append(y)
                if legal_route(l_temp):
                    splited_routes.append(tuple(l_temp))
        return splited_routes

def grp_avg_distance(group):
    """calculates the average distance in km for a list of routes

    Args:
        group (list): a list of routes
    
    Returns:
        float: returns the average distance in km of the routes passed in group argument
    """
    return sum([route.distance for route in group]) / len(group)

def get_min_distance_grp(groups):
    """returns a group of routes with the minimum average distance

    Args:
        groups (list): a list of tuples that contains routes
    
    Returns:
        tuple: returns a tuple of the routes with the average minimum distance
    """
    min_grp = groups[0]
    min_distance = grp_avg_distance(groups[0])
    if len(groups) < 2:
        return min_grp
    for group in groups[1:]:
        avg_distance = grp_avg_distance(group)
        if avg_distance < min_distance:
            min_grp = group
            min_distance = avg_distance
    return min_grp

def show_grp(group, zoom=7):
    """displays a tuple of routes on a map 

    Args:
        group (tuple): a tuple of routes
        zoom (int): the initial zoom on the map (defaults as 7)
    
    Returns:
        Map: returns a Map object featuring a all the routes passed through argument group
    """
    start = list(group[0].locations.values())[0].coord # get first location coord
    m = folium.Map(location = [*start], zoom_start=zoom) # create map object
    for route in group:
        route.display(m=m) # add routes to existing object map
    return m

def export_grp_csv(filename, group):
    """exports a csv file based on the routes found in the group argument

    Args:
        filename (str): filepath for the exported csv file
        group (tuple): a tuple of routes, contains all the data to be exported 
    """
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['fname', 'lname', 'age', 'gender', 'country', 'city', 'street','full address', 'group id'])
        for id_num, route in enumerate(group, 1):
            for person in route.locations:
                writer.writerow([person.fname,
                                    person.lname,
                                    person.age,
                                    person.gender,
                                    person.address.country,
                                    person.address.city, 
                                    person.address.street,
                                    person.address.full_address,
                                    id_num])
        print(f'wrote csv file: {filename} succefully')
            

# Loading Information and Runing the progrum

In [5]:
# load csv data 
people = get_people_from_csv('people_Un.csv')


examining the info

In [6]:
from prettytable import PrettyTable
    
table = PrettyTable()

table.field_names = ["First Name", "Last Name", "Age", "Gender", "Country", "City", "Street", "Full Address"]

for person in people:
    table.add_row([person.fname, 
               person.lname, 
               person.age, 
               person.gender,
               person.address.country, 
               person.address.city, 
               person.address.street, 
               person.address.full_address])
print(table)

+------------+-----------+-----+--------+---------+----------------+--------------------+-------------------------------------------+
| First Name | Last Name | Age | Gender | Country |      City      |       Street       |                Full Address               |
+------------+-----------+-----+--------+---------+----------------+--------------------+-------------------------------------------+
|   Assaf    |   Hazan   |  30 |   M    |  Israel | Tel Aviv-Yafo  |    Beit Hilel 1    |    Israel ,Tel Aviv-Yafo ,Beit Hilel 1    |
|   Daniel   | Premisler |  33 |   M    |  Israel |  Petah Tikva   |     Ahad HaAm      |      Israel ,Petah Tikva ,Ahad HaAm       |
|    Adam    |   Efrati  |  34 |   M    |  Israel |   Bnei Brak    |    Dov Gruner 2    |      Israel ,Bnei Brak ,Dov Gruner 2      |
|    Serj    |  Itzhakov |  34 |   M    |  Israel | Tel Aviv-Yafo  |       Irus 2       |       Israel ,Tel Aviv-Yafo ,Irus 2       |
|    Alon    |  Shomron  |  32 |   M    |  Israel | Rishon LeZ

we can also show the different locations on a map

In [7]:
marker = people[0]
m = marker.display(zoom=10)
for person in people[1:]:
    person.display(m=m)
    
display(m)

Now we want to create routes from the different locations and find the optimal group

In [8]:
# create routes from people and split them to groups
groups_ = make_groups(people)

# get group with min distance
min_grp = get_min_distance_grp(groups_)

Lets print the group with the minimum distance (optimal group)

In [9]:
for i, route in enumerate(min_grp, 1):
    print(f'the {i} route:')
    print(route)

the 1 route:
numer of locations: 2
personal info: Daniel Premisler ,33 ,M ; address: Israel ,Petah Tikva ,Ahad HaAm 
personal info: Dana Saar ,25 ,F     ; address: Israel ,Kadima - Zoran ,HaHistadrut 
the 2 route:
numer of locations: 2
personal info: Alon Shomron ,32 ,M  ; address: Israel ,Rishon LeZion ,Katznelson Berl 26
personal info: Jenia Premisler ,60 ,M ; address: Israel ,Rishon LeZion ,HaNurit 1
the 3 route:
numer of locations: 2
personal info: Olla Premisler ,38 ,F ; address: Israel ,Haifa ,Keren HaYesod 
personal info: Noam Saar ,30 ,M     ; address: Israel ,Netanya ,Moriya 
the 4 route:
numer of locations: 3
personal info: Assaf Hazan ,30 ,M   ; address: Israel ,Tel Aviv-Yafo ,Beit Hilel 1
personal info: Adam Efrati ,34 ,M   ; address: Israel ,Bnei Brak ,Dov Gruner 2
personal info: Serj Itzhakov ,34 ,M ; address: Israel ,Tel Aviv-Yafo ,Irus 2


We can now look at the routes on a map

In [10]:
show_grp(min_grp)

Finally we can export a csv with personal information along with an id per each route

In [11]:
export_grp_csv('group.csv' ,min_grp)

wrote csv file: group.csv succefully


adding another person and address - without a file

In [12]:
michal_address = Address(country='ישראל', city='תל אביב', street='ריינס 31')
michal = Person('michal', 'halevi', '29', 'f', address=michal_address)

we can display and customize some properties of the marker

In [13]:
# display a person on a map with custom popup and tooltip and marker color
michal.display(c='red', m=m)

In [14]:
# get another person from a list of person classes
assaf = people[0]

In [15]:
# creating a route between two persons and showint it on a map
r= Route(michal, assaf)
r.display()

lets now add the new person to our list from the csv file

In [16]:
people.append(michal)

Once again we find the optimal group but this time with another location
we also print the new group

In [17]:
# create routes from people and split them to groups
groups_ = make_groups(people)

# get group with min distance
min_grp = get_min_distance_grp(groups_)

# printing the optimal group
for i, route in enumerate(min_grp, 1):
    print(f'the {i} route:')
    print(route)

the 1 route:
numer of locations: 2
personal info: Assaf Hazan ,30 ,M   ; address: Israel ,Tel Aviv-Yafo ,Beit Hilel 1
personal info: Serj Itzhakov ,34 ,M ; address: Israel ,Tel Aviv-Yafo ,Irus 2
the 2 route:
numer of locations: 2
personal info: Daniel Premisler ,33 ,M ; address: Israel ,Petah Tikva ,Ahad HaAm 
personal info: Dana Saar ,25 ,F     ; address: Israel ,Kadima - Zoran ,HaHistadrut 
the 3 route:
numer of locations: 2
personal info: Adam Efrati ,34 ,M   ; address: Israel ,Bnei Brak ,Dov Gruner 2
personal info: Michal Halevi ,29 ,F ; address: Israel ,Tel Aviv-Yafo ,Reines 31
the 4 route:
numer of locations: 2
personal info: Alon Shomron ,32 ,M  ; address: Israel ,Rishon LeZion ,Katznelson Berl 26
personal info: Jenia Premisler ,60 ,M ; address: Israel ,Rishon LeZion ,HaNurit 1
the 5 route:
numer of locations: 2
personal info: Olla Premisler ,38 ,F ; address: Israel ,Haifa ,Keren HaYesod 
personal info: Noam Saar ,30 ,M     ; address: Israel ,Netanya ,Moriya 


we can once again display our routes

In [18]:
show_grp(min_grp)